In [1]:
from langgraph_sdk import get_client
client = get_client(url="http://127.0.0.1:2024")

In [2]:
%pip install langchain-core

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
     ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
     ------------------------ --------------- 41.0/68.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------- 68.0/68.0 kB 1.3 MB/s eta 0:00:00
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp311-cp311-win_amd64.whl.metadata (3.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached charset_normalizer-3.4.2-cp311-c


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_core.messages import HumanMessage

In [6]:
initial_input = {"messages": HumanMessage(content='Multiply 4 and 5')}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread['thread_id'],
    assistant_id="timetravel",
    input=initial_input,
    stream_mode="updates"
):
    if chunk.data:
        assistant_node = chunk.data.get('assistant', {}).get('messages',[])
        tool_node = chunk.data.get('tools', {}).get('messages',[])
        if assistant_node:
            print("-"*20 + "Assistant node" + "-" * 20)
            print(assistant_node[-1])
        elif tool_node:
            print("-"*20 + "Tool node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant node--------------------
{'content': '', 'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 5.0}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 5, 'total_token_count': 104, 'prompt_tokens_details': [{'modality': 1, 'token_count': 99}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001'}, 'type': 'ai', 'name': None, 'id': 'run--be79989a-7a30-4069-b0bc-f5647011d915-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 4.0, 'b': 5.0}, 'id': '22552a26-2018-4516-b0b5-89fe86ee67f0', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 99, 'output_tokens': 5, 'total_tokens': 104}}
--------------------Tool node-------------

In [7]:
# get the history for replaying

states = await client.threads.get_history(thread['thread_id'])

In [8]:
states

[{'values': {'messages': [{'content': 'Multiply 4 and 5',
     'additional_kwargs': {},
     'response_metadata': {},
     'type': 'human',
     'name': None,
     'id': '90abfcd8-8999-4d73-831f-fd33acfb2c69',
     'example': False},
    {'content': '',
     'additional_kwargs': {'function_call': {'name': 'multiply',
       'arguments': '{"a": 4.0, "b": 5.0}'}},
     'response_metadata': {'is_blocked': False,
      'safety_ratings': [],
      'usage_metadata': {'prompt_token_count': 99,
       'candidates_token_count': 5,
       'total_token_count': 104,
       'prompt_tokens_details': [{'modality': 1, 'token_count': 99}],
       'candidates_tokens_details': [{'modality': 1, 'token_count': 5}],
       'thoughts_token_count': 0,
       'cached_content_token_count': 0,
       'cache_tokens_details': []},
      'finish_reason': 'STOP',
      'model_name': 'gemini-2.0-flash-001'},
     'type': 'ai',
     'name': None,
     'id': 'run--be79989a-7a30-4069-b0bc-f5647011d915-0',
     'example'

In [9]:
to_replay = states[-2]
to_replay

{'values': {'messages': [{'content': 'Multiply 4 and 5',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '90abfcd8-8999-4d73-831f-fd33acfb2c69',
    'example': False}]},
 'next': ['assistant'],
 'tasks': [{'id': 'c09ab561-a3e4-dea5-614c-0810af31e283',
   'name': 'assistant',
   'path': ['__pregel_pull', 'assistant'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': {'messages': [{'content': '',
      'additional_kwargs': {'function_call': {'name': 'multiply',
        'arguments': '{"a": 4.0, "b": 5.0}'}},
      'response_metadata': {'is_blocked': False,
       'safety_ratings': [],
       'usage_metadata': {'prompt_token_count': 99,
        'candidates_token_count': 5,
        'total_token_count': 104,
        'prompt_tokens_details': [{'modality': 1, 'token_count': 99}],
        'candidates_tokens_details': [{'modality': 1, 'token_count': 5}],
        'thoughts_token_count': 0,
     

In [ ]:
# to replay give input as None, pas the thread and checkpoint id

async for chunk in client.runs.stream(
    thread_id=thread['thread_id'],
    assistant_id="timetravel",
    input=None,
    stream_mode="values",
    checkpoint_id=to_replay['checkpoint_id']
):
    print(f"Event type: {chunk.event}")
    print(chunk.data)
    print("\n\n")

Event type: metadata
{'run_id': '1f049b41-cf3b-63f5-91fd-1e605850d44a', 'attempt': 1}



Event type: values
{'messages': [{'content': 'Multiply 4 and 5', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '90abfcd8-8999-4d73-831f-fd33acfb2c69', 'example': False}]}



Event type: values
{'messages': [{'content': 'Multiply 4 and 5', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '90abfcd8-8999-4d73-831f-fd33acfb2c69', 'example': False}, {'content': '', 'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 5.0}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 5, 'total_token_count': 104, 'prompt_tokens_details': [{'modality': 1, 'token_count': 99}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_de

In [12]:
async for chunk in client.runs.stream(
    thread_id=thread['thread_id'],
    assistant_id="timetravel",
    input=None,
    stream_mode="updates",
    checkpoint_id=to_replay['checkpoint_id']
):
    if chunk.data:
        assistant_node = chunk.data.get('assistant', {}).get('messages',[])
        tool_node = chunk.data.get('tools', {}).get('messages',[])
        if assistant_node:
            print("-"*20 + "Assistant node" + "-" * 20)
            print(assistant_node[-1])
        elif tool_node:
            print("-"*20 + "Tool node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant node--------------------
{'content': '', 'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 5.0}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 5, 'total_token_count': 104, 'prompt_tokens_details': [{'modality': 1, 'token_count': 99}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001'}, 'type': 'ai', 'name': None, 'id': 'run--87ddb037-291c-40b5-bacc-deceb8f0ab50-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 4.0, 'b': 5.0}, 'id': 'd7a667f4-843e-4b9f-8f77-48a1cf1662b1', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 99, 'output_tokens': 5, 'total_tokens': 104}}
--------------------Tool node-------------

# Forking

In [13]:
initial_input = {"messages": HumanMessage(content='Multiply 4 and 5')}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread['thread_id'],
    assistant_id="timetravel",
    input=initial_input,
    stream_mode="updates"
):
    if chunk.data:
        assistant_node = chunk.data.get('assistant', {}).get('messages',[])
        tool_node = chunk.data.get('tools', {}).get('messages',[])
        if assistant_node:
            print("-"*20 + "Assistant node" + "-" * 20)
            print(assistant_node[-1])
        elif tool_node:
            print("-"*20 + "Tool node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant node--------------------
{'content': '', 'additional_kwargs': {'function_call': {'name': 'multiply', 'arguments': '{"a": 4.0, "b": 5.0}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 5, 'total_token_count': 104, 'prompt_tokens_details': [{'modality': 1, 'token_count': 99}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001'}, 'type': 'ai', 'name': None, 'id': 'run--95bab769-74c2-4b18-b8da-a1e53ba6d527-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 4.0, 'b': 5.0}, 'id': '50bc077b-20bd-4d89-b43f-a851c778e92c', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 99, 'output_tokens': 5, 'total_tokens': 104}}
--------------------Tool node-------------

In [14]:
states = await client.threads.get_history(thread_id=thread['thread_id'])
to_fork = states[-2]
to_fork['values']

{'messages': [{'content': 'Multiply 4 and 5',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': '04827649-7235-49f5-8223-7aeb66b1d423',
   'example': False}]}

In [15]:
to_fork['values']['messages'][0]['id']

'04827649-7235-49f5-8223-7aeb66b1d423'

In [16]:
to_fork['next']

['assistant']

In [17]:
to_fork['checkpoint_id']

'1f049b47-5a6c-6e6a-8000-ee2e1bfd62a3'

In [18]:
forked_input = {"messages": HumanMessage(
    content="Add 4 and 5",
    id=to_fork['values']['messages'][0]['id']
)}

In [19]:
forked_config = await client.threads.update_state(
    thread_id=thread['thread_id'],
    values=forked_input,
    checkpoint_id=to_fork['checkpoint_id']
)

In [20]:
states = await client.threads.get_history(thread['thread_id'])
states[0]

{'values': {'messages': [{'content': 'Add 4 and 5',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '04827649-7235-49f5-8223-7aeb66b1d423',
    'example': False}]},
 'next': ['assistant'],
 'tasks': [{'id': '1b1da0e1-34b1-cb50-262a-ba4e82a4f3d3',
   'name': 'assistant',
   'path': ['__pregel_pull', 'assistant'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': None}],
 'metadata': {'graph_id': 'timetravel',
  'thread_id': 'eb1ad0d5-bff5-41c5-b0a2-755f686797d5',
  'checkpoint_id': '1f049b47-5a6c-6e6a-8000-ee2e1bfd62a3',
  'checkpoint_ns': '',
  'user-agent': 'langgraph-sdk-py/0.1.70',
  'x-request-id': '6688be35-cb35-48ae-8cd4-83dfe825285e',
  'langgraph_auth_user': None,
  'langgraph_auth_user_id': '',
  'langgraph_auth_permissions': [],
  'langgraph_request_id': '6688be35-cb35-48ae-8cd4-83dfe825285e',
  'assistant_id': '6fa3be18-20fb-599c-9056-812c324a3a39',
  'user_id': '',
  'creat

In [22]:
async for chunk in client.runs.stream(
    thread['thread_id'],
    assistant_id="timetravel",
    input=None,
    stream_mode="updates",
    checkpoint_id=forked_config['checkpoint_id']
):
    if chunk.data:
        assistant_node = chunk.data.get('assistant', {}).get('messages',[])
        tool_node = chunk.data.get('tools', {}).get('messages',[])
        if assistant_node:
            print("-"*20 + "Assistant node" + "-" * 20)
            print(assistant_node[-1])
        elif tool_node:
            print("-"*20 + "Tool node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant node--------------------
{'content': '', 'additional_kwargs': {'function_call': {'name': 'add', 'arguments': '{"a": 4.0, "b": 5.0}'}}, 'response_metadata': {'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 99, 'candidates_token_count': 5, 'total_token_count': 104, 'prompt_tokens_details': [{'modality': 1, 'token_count': 99}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 5}], 'thoughts_token_count': 0, 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -5.276140291243791e-06, 'model_name': 'gemini-2.0-flash-001'}, 'type': 'ai', 'name': None, 'id': 'run--d3aa8da5-9a91-40bb-9658-e68cf0c4a7d7-0', 'example': False, 'tool_calls': [{'name': 'add', 'args': {'a': 4.0, 'b': 5.0}, 'id': '2363823d-803b-463a-b631-194e79d62c22', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 99, 'output_tokens': 5, 'total_tokens': 104}}
------------